# HiPPO Matrices
---

## Table of Contents
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
    * [Translated Legendre (LegT)](#translated-legendre-legt)
        * [LegT](#legt)
        * [LMU](#lmu)
    * [Translated Laguerre (LagT)](#translated-laguerre-lagt)
    * [Scaled Legendre (LegS)](#scaled-legendre-legs)
    * [Fourier Basis](#fourier-basis)
        * [Fourier Recurrent Unit (FRU)](#fourier-recurrent-unit-fru)
        * [Truncated Fourier (FouT)](#truncated-fourier-fout)
        * [Fourier With Decay (FourD)](#fourier-with-decay-fourd)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
    * [Testing Forward Euler on GBT matrices](#testing-forward-euler-transform-for-lti-and-lsi)
    * [Testing Backward Euler on GBT matrices](#testing-backward-euler-transform-for-lti-and-lsi-on-legs-matrices)
    * [Testing Bidirectional on GBT matrices](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on GBT matrices](#testing-zoh-transform-for-lti-and-lsi-on-legs-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
    * [Testing Forward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-forward-euler-transform)
    * [Testing Backward Euler on HiPPO Operators](#testing-lti-and-lsi-operators-with-backward-euler-transform)
    * [Testing Bidirectional on HiPPO Operators](#testing-lti-and-lsi-operators-with-bidirectional-transform)
    * [Testing ZOH on HiPPO Operators](#testing-lti-and-lsi-operators-with-zoh-transform)
---


## Load Packages

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../../../"))
print(f"module_path: {module_path}")
if module_path not in sys.path:
    print(f"Adding {module_path} to sys.path")
    sys.path.append(module_path)

module_path: /home/beegass/Documents/Coding/s4mer


In [2]:
## import packages
import math

import jax
import jax.numpy as jnp
import requests
from flax import linen as jnn
from jax.nn.initializers import lecun_normal, uniform
from jax.numpy.linalg import eig, inv, matrix_power
from jax.scipy.signal import convolve

from jaxtyping import Array, Float, Float16, Float32, Float64
from typing import Callable, List, Optional, Tuple, Any, Union

from scipy import linalg as la
from scipy import signal
from scipy import special as ss

from src.data.process import moving_window, rolling_window

# import modules
from src.models.hippo.gu_transition import GuTransMatrix
from src.models.hippo.transition import TransMatrix
from src.models.hippo.gu_hippo import gu_HiPPO_LSI, gu_HiPPO_LTI
from src.models.hippo.hippo import HiPPOLTI, HiPPOLSI
from src.models.hippo.unroll import (
    basis,
    measure,
    variable_unroll_matrix,
    variable_unroll_matrix_sequential,
)
from src.utils.ops import genlaguerre, legendre_polynomial

print(jax.devices())
print(f"The Device: {jax.lib.xla_bridge.get_backend().platform}")

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
The Device: gpu


In [3]:
from functools import partial
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import functorch
import einops

print(f"MPS enabled: {torch.backends.mps.is_available()}")

MPS enabled: False


In [4]:
torch.set_printoptions(linewidth=150)
np.set_printoptions(linewidth=150)
jnp.set_printoptions(linewidth=150)

In [5]:
seed = 1701
key = jax.random.PRNGKey(seed)

In [6]:
num_copies = 10
subkeys = jax.random.split(key, num=num_copies)
key = subkeys[0]

In [7]:
def random_16_input(key_generator, batch_size=16, data_size=784, input_size=28):
    # x = jax.random.randint(key_generator, (batch_size, data_size), 0, 255)
    x = jax.random.uniform(key_generator, (batch_size, data_size))
    return np.asarray(jax.vmap(moving_window, in_axes=(0, None))(x, input_size))

## Test HiPPO Reconstruction

In [8]:
def test_hippo_reconstruction(
    hippo, gu_hippo, random_input, key, s_or_t="lti", print_all=False
):
    x_tensor = torch.tensor(random_input, dtype=torch.float32)
    x_jnp = jnp.asarray(x_tensor, dtype=jnp.float32)  # convert torch array to jax array

    # My Implementation
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"----------------------------My {s_or_t} Implementation Outputs----------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    params = hippo.init(key, f=x_jnp)
    hippo = hippo.bind(params)
    c = hippo.__call__(f=x_jnp)
    y = hippo.reconstruct(c)
    print(f"c shape:\n {c.shape}")

    mse = lambda y_hat, y: jnp.mean((y_hat - y) ** 2)
    batch_mse = jax.vmap(mse, in_axes=(0, 0))

    # Gu's HiPPO LegS
    if print_all:
        print(
            f"------------------------------------------------------------------------------------------"
        )
        print(
            f"---------------------------Gu's {s_or_t} Implementation Outputs---------------------------"
        )
        print(
            f"------------------------------------------------------------------------------------------"
        )
    x_tensor = torch.moveaxis(x_tensor, 0, 1)
    GU_c_s, GU_c_k = gu_hippo(x_tensor, fast=False)
    # print(f"GU_c_k shape:\n{GU_c_k.shape}")

    gu_y = None
    if s_or_t == "lsi":
        print(f"GU_c_s shape:\n{GU_c_s.shape}")
        gu_y = gu_hippo.reconstruct(GU_c_s)
    elif s_or_t == "lti":
        print(f"GU_c_k shape:\n{GU_c_k.shape}")
        gu_y = gu_hippo.reconstruct(GU_c_k)
    else:
        raise ValueError(
            f"s_or_t must be either 'lsi' or 'lti'. s_or_t is currently set to: {s_or_t}"
        )

    gu_c = jnp.asarray(GU_c_k, dtype=jnp.float32)  # convert torch array to jax array
    gu_c_s = jnp.asarray(GU_c_s, dtype=jnp.float32)  # convert torch array to jax array
    gu_y = jnp.asarray(gu_y, dtype=jnp.float32)  # convert torch array to jax array

    x_tensor = jnp.asarray(
        x_tensor, dtype=jnp.float32
    )  # convert torch array to jax array
    x_tensor = jnp.moveaxis(x_tensor, 0, 1)

    flag = True
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):

            check = None
            if s_or_t == "lsi":
                if print_all:
                    print(f"c_k[{i},{j},:,:]:\n{c[i,j,:,:]}")
                    print(f"gu_cs[{i},{j},:,:]:\n{gu_c_s[i,j,:,:]}")
                check = jnp.allclose(
                    c[i, j, :, :], gu_c_s[i, j, :, :], rtol=1e-04, atol=1e-04
                )
            elif s_or_t == "lti":
                if print_all:
                    print(f"c_k[{i},{j},:]:\n{c[i,j,:]}")
                    print(f"gu_cs[{i},{j},:]:\n{gu_c[i,j,:]}")
                check = jnp.allclose(c[i, j, :], gu_c[i, j, :], rtol=1e-04, atol=1e-04)
                
            if print_all:
                print(f"check: {check}\n")
                
            if check == False:
                flag = False
    if not print_all:
        print(f"\n---------------------------------------------")
        print(f"---------- The Test Passed: {flag} ----------")
        print(f"---------------------------------------------\n")

    print(f"y shape:\n {y.shape}")
    print(f"gu_y shape:\n {gu_y.shape}")

    if print_all:
        print(f"y:\n {y}")
        print(f"gu_y:\n {gu_y}")

    time_allclose = jax.vmap(jnp.allclose, in_axes=(0, 0, None, None))
    batch_time_allclose = jax.vmap(time_allclose, in_axes=(0, 0, None, None))
    print(f"Test y:\n{batch_time_allclose(y, gu_y, 1e-04, 1e-04)}\n")
    print(f"y Loss:\n {batch_mse(gu_y, x_tensor)}\n")
    print(f"gu's y Loss:\n {batch_mse(y, x_jnp)}\n\n\n")

In [9]:
def test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=False
):
    # N = 256
    # L = 128

    batch_size = 2
    data_size = 64
    input_size = 1

    N = 50
    L = data_size

    x_jnp = random_16_input(
        key_generator=subkeys[4],
        batch_size=batch_size,
        data_size=data_size,
        input_size=input_size,
    )
    x_np = np.asarray(x_jnp)

    x = torch.tensor(x_np, dtype=torch.float32)

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate Gu's HiPPOs ---------------------------
    # ----------------------------------------------------------------------------------

    print(f"Creating Gu's HiPPO-{the_measure} LTI model with {alpha} transform")
    gu_hippo_lti = gu_HiPPO_LTI(
        N=N,
        method=the_measure,
        dt=1.0,
        T=L,
        discretization=discretization,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        c=0.0,
    )  # The Gu's

    if the_measure == "legs":
        print(f"Creating Gu's HiPPO-{the_measure} LSI model with {alpha} transform")
        gu_hippo_lsi = gu_HiPPO_LSI(
            N=N,
            method=the_measure,
            max_length=L,
            discretization=discretization,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
        )  # The Gu's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Instantiate My HiPPOs -----------------------------
    # ----------------------------------------------------------------------------------
    print(f"\nTesting BRYANS HiPPO-{the_measure} model")

    matrices = TransMatrix(
        N=N,
        measure=the_measure,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        dtype=jnp.float32,
    )

    A = matrices.A
    B = matrices.B

    print(f"Creating HiPPO-{the_measure} LTI model with {alpha} transform")
    hippo_lti = HiPPOLTI(
        N=N,
        step_size=1.0,
        lambda_n=lambda_n,
        alpha=0.0,
        beta=1.0,
        GBT_alpha=alpha,
        measure=the_measure,
        basis_size=L,
        dtype=jnp.float32,
        unroll=False,
    )  # Bryan's

    if the_measure == "legs":
        print(f"Creating HiPPO-{the_measure} LSI model with {alpha} transform")
        hippo_lsi = HiPPOLSI(
            N=N,
            max_length=L,
            step_size=1.0,
            lambda_n=lambda_n,
            alpha=0.0,
            beta=1.0,
            GBT_alpha=alpha,
            measure=the_measure,
            dtype=jnp.float32,
            unroll=True,
        )  # Bryan's

    # ----------------------------------------------------------------------------------
    # ------------------------------ Test HiPPO Operators ------------------------------
    # ----------------------------------------------------------------------------------

    print(f"Bryan's Coeffiecients for {alpha} LTI HiPPO-{the_measure}")

    test_hippo_reconstruction(
        hippo=hippo_lti,
        gu_hippo=gu_hippo_lti,
        random_input=x_np,
        key=subkeys[5],
        s_or_t="lti",
        print_all=print_all,
    )

    if the_measure == "legs":
        print(f"\n\nBryan's Coeffiecients for {alpha} LSI HiPPO-{the_measure}")

        test_hippo_reconstruction(
            hippo=hippo_lsi,
            gu_hippo=gu_hippo_lsi,
            random_input=x_np,
            key=subkeys[6],
            s_or_t="lsi",
            print_all=print_all,
        )

    print(f"end of test for HiPPO-{the_measure} model")

## Navigation To Table Of Contents
---
* [Table Of Contents](#table-of-contents)
* [Loading In Necessary Packages](#load-packages)
* [Instantiate The HiPPO Matrix](#instantiate-the-hippo-matrix)
* [Gu's Linear Time Invariant (LTI) HiPPO Operator](#gus-hippo-legt-operator)
* [Gu's Scale invariant (LSI) HiPPO Operator](#gus-scale-invariant-hippo-legs-operator)
* [Implementation Of General HiPPO Operator](#implementation-of-general-hippo-operator)
* [Test Generalized Bilinear Transform and Zero Order Hold Matrices](#test-generalized-bilinear-transform-and-zero-order-hold-matrices)
* [Testing HiPPO Operators](#test-hippo-operators)
---

In [10]:
print_all = True

### Testing (LTI and LSI) Operators With Forward Euler Transform

#### LegS

In [11]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.0 transform
Creating Gu's HiPPO-legs LSI model with 0.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.0 transform
Creating HiPPO-legs LSI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legs
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2,

#### LegT

In [12]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-legt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------


/home/beegass/.cache/pypoetry/virtualenvs/s4mer-pkg-jZnBSgjq-py3.8/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:4983: RuntimeWarning: overflow encountered in cast
  return binary_op(*args)


LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan]
gu_cs[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan]
c_k[1,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan na

#### LMU

In [13]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lmu
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

#### LagT

In [14]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-lagt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.5616925e-01  5.3887522e-01  5.3389108e-01  2.8113365e-02 -1.3089989e+00 -3.314

#### FRU

In [15]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fru
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

#### FouT

In [16]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-fout
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

#### FouD

In [17]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=0.0, discretization=0.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.0 transform
Bryan's Coeffiecients for 0.0 LTI HiPPO-foud
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

### Testing (LTI and LSI) Operators With Backward Euler Transform

#### LegS

In [18]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 1.0 transform
Creating Gu's HiPPO-legs LSI model with 1.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 1.0 transform
Creating HiPPO-legs LSI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legs
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2,

#### LegT

In [19]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 1.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-legt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.6734447e-01  8.7768681e-02  1.5264817e-02  1.0787459e-03 -4.3758249e-05 -1.341

#### LMU

In [20]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 1.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lmu
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.6736534e-01 -1.5200797e-01  3.4119863e-02 -2.8613273e-03 -1.3249388e-04  4.7848145

#### LagT

In [21]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 1.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-lagt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.1594226e-01  1.1028503e-01  4.6868794e-02  3.0574476e-02  2.3672692e-02  1.614

#### FRU

In [22]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 1.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fru
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[6.86941385e-01 3.06169131e-05 3.47787812e-02 7.08667636e-02 9.09158960e-03 4.09489721

#### FouT

In [23]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 1.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-fout
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.67853677e-01 -6.07839729e-06  1.46930125e-02  6.79084510e-02  3.63567378e-03  

#### FouD

In [24]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=1.0, discretization=1.0, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 1.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 1.0 transform
Bryan's Coeffiecients for 1.0 LTI HiPPO-foud
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[6.23453200e-01 1.56405640e-05 2.27625053e-02 5.36189079e-02 5.98547794e-03 3.0457

### Testing (LTI and LSI) Operators With Bidirectional Transform

#### LegS

In [25]:
test_reconstruction(
    the_measure="legs", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legs LTI model with 0.5 transform
Creating Gu's HiPPO-legs LSI model with 0.5 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 0.5 transform
Creating HiPPO-legs LSI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legs
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2,

#### LegT

In [26]:
test_reconstruction(
    the_measure="legt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-legt LTI model with 0.5 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-legt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.5928383e-01  2.0949930e-01 -2.3842663e-02 -4.8999500e-02 -1.8362312e-01  3.615

#### LMU

In [27]:
test_reconstruction(
    the_measure="lmu", lambda_n=2.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lmu LTI model with 0.5 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lmu
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.59317267e-01 -3.62854540e-01 -5.33035994e-02  1.29634097e-01 -5.50831497e-01 -1.19

#### LagT

In [28]:
test_reconstruction(
    the_measure="lagt", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-lagt LTI model with 0.5 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-lagt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.88789809e-01  1.99238360e-01  1.60363704e-01  1.52378306e-01  1.19987756e-01  

#### FRU

In [29]:
test_reconstruction(
    the_measure="fru", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fru LTI model with 0.5 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fru
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 5.7321346e-01  6.8542172e-06  1.4166957e-01  3.6353928e-01  4.7893640e-02  1.4053437

#### FouT

In [30]:
test_reconstruction(
    the_measure="fout", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-fout LTI model with 0.5 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-fout
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.66803968e-01  3.18401471e-05  4.57847565e-02  2.26757795e-01 -3.42129380e-01 -

#### FouD

In [31]:
test_reconstruction(
    the_measure="foud", lambda_n=1.0, alpha=0.5, discretization=0.5, print_all=print_all
)

Creating Gu's HiPPO-foud LTI model with 0.5 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 0.5 transform
Bryan's Coeffiecients for 0.5 LTI HiPPO-foud
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 5.8978140e-01  4.6341529e-06  8.2808293e-02  3.2692721e-01  3.4393564e-02  7.902

### Testing (LTI and LSI) Operators With ZOH Transform

#### LegS

In [32]:
test_reconstruction(
    the_measure="legs",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legs LTI model with 2.0 transform
Creating Gu's HiPPO-legs LSI model with 2.0 transform

Testing BRYANS HiPPO-legs model
Creating HiPPO-legs LTI model with 2.0 transform
Creating HiPPO-legs LSI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legs
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2,

#### LegT

In [33]:
test_reconstruction(
    the_measure="legt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-legt LTI model with 2.0 transform

Testing BRYANS HiPPO-legt model
Creating HiPPO-legt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-legt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.5445108e-01  3.0463033e-03 -3.9678589e-03  4.6666283e-03 -5.2409638e-03  5.710

#### LMU

In [34]:
test_reconstruction(
    the_measure="lmu",
    lambda_n=2.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lmu LTI model with 2.0 transform

Testing BRYANS HiPPO-lmu model
Creating HiPPO-lmu LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lmu
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.54450369e-01 -5.27745625e-03 -8.87205079e-03 -1.23465974e-02 -1.57232881e-02 -1.89

#### LagT

In [35]:
test_reconstruction(
    the_measure="lagt",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-lagt LTI model with 2.0 transform

Testing BRYANS HiPPO-lagt model
Creating HiPPO-lagt LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-lagt
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 7.73455560e-01  1.61825404e-01  9.47052240e-02  6.14058338e-02  1.83721241e-02 -

#### FRU

In [36]:
test_reconstruction(
    the_measure="fru",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fru LTI model with 2.0 transform

Testing BRYANS HiPPO-fru model
Creating HiPPO-fru LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fru
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 6.88785017e-01  0.00000000e+00 -3.17948544e-03  2.42512718e-01  2.89788493e-03  1.77

#### FouT

In [37]:
test_reconstruction(
    the_measure="fout",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-fout LTI model with 2.0 transform

Testing BRYANS HiPPO-fout model
Creating HiPPO-fout LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-fout
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 9.5464873e-01  0.0000000e+00 -2.1553200e-03 -4.7074827e-05 -2.1569883e-03 -9.245

#### FouD

In [38]:
test_reconstruction(
    the_measure="foud",
    lambda_n=1.0,
    alpha=2.0,
    discretization="zoh",
    print_all=print_all,
)

Creating Gu's HiPPO-foud LTI model with 2.0 transform

Testing BRYANS HiPPO-foud model
Creating HiPPO-foud LTI model with 2.0 transform
Bryan's Coeffiecients for 2.0 LTI HiPPO-foud
------------------------------------------------------------------------------------------
----------------------------My lti Implementation Outputs----------------------------
------------------------------------------------------------------------------------------
LTI eval_matrix.shape: (64, 50)
c.shape: (2, 1, 50)
c shape:
 (2, 1, 50)
------------------------------------------------------------------------------------------
---------------------------Gu's lti Implementation Outputs---------------------------
------------------------------------------------------------------------------------------
GU_c_k shape:
torch.Size([2, 1, 50])
gu LTI eval_matrix.shape: torch.Size([64, 50])
c.shape: torch.Size([2, 1, 50])
c_k[0,0,:]:
[ 6.42800450e-01  0.00000000e+00 -7.81296403e-04  1.28640324e-01  5.63864771e-04  